In [ ]:
#https://github.com/AliTavassoly/DL-Bulletproof

In [1]:
import numpy as np
import hashlib

# compute product of (g_i ^ x_i) mod q
def vector_exponent(g, x, q):
    c = 1
    for i in range(len(x)):
        c *= pow(int(g[i]), int(x[i]), q)
        c %= q
    return c

# hash string s module p
def get_hash(s, p):
    a = int(hashlib.sha256(s.encode()).hexdigest(), 16) % p
    while a == 0:
        s += '1'
        a = int(hashlib.sha256(s.encode()).hexdigest(), 16) % p
    return a

In [2]:
class Prover:
    # q is the order of larger group
    # p is the order of the cyclic subgroup
    def __init__(self, p, q):
        self.p = p
        self.q = q

    def get_initial_commitment(self, x, g):
        c = 1
        for i in range(len(x)):
            c *= pow(int(g[i]), int(x[i]), self.q)
            c %= self.q
        return c

    def commit(self, g, x):
        g = g.copy()
        x = x.copy()

        initial_c = c = self.get_initial_commitment(x, g)

        vRvector = []
        vLvector = []

        while len(x) > 1:
            mid_point = len(x) // 2

            xL = x[:mid_point]
            xR = x[mid_point:]

            gL = g[:mid_point]
            gR = g[mid_point:]

            vL = vector_exponent(gR, xL, self.q)
            vR = vector_exponent(gL, xR, self.q)
            vLvector.append(vL)
            vRvector.append(vR)

            combined = f"{c}{vL}{vR}"
            a = get_hash(combined, self.p)

            c = c * pow(int(vL), a * a % self.p, self.q) % self.q * pow(int(vR), pow(a * a, -1, self.p), self.q) % self.q

            x = a * xL + pow(a, -1, self.p) * xR
            x %= self.p

            modL = np.vectorize(lambda x: pow(int(x), pow(a, -1, self.p), self.q))
            modR = np.vectorize(lambda x: pow(int(x), a, self.q))
            g = np.mod(modL(gL) * modR(gR), self.q)

        return {'initial_c': initial_c, 'vLvector': vLvector, 'vRvector': vRvector, 'final_x': x[0]}

In [3]:
class Verifier:
    # q is the order of larger group
    # p is the order of the cyclic subgroup
    def __init__(self, p, q):
        self.p = p
        self.q = q

    def extract_commitment(self, com_string):
        return (com_string['initial_c'],
                com_string['vLvector'],
                com_string['vRvector'],
                com_string['final_x'])

    def verify_commitment(self, g, com_string):
        g = g.copy()

        (c, vLvector, vRvector, final_x) = self.extract_commitment(com_string)

        assert(len(vLvector) == len(vRvector))

        rounds_count = len(vLvector)
        for i in range(rounds_count):
            vL = vLvector[i]
            vR = vRvector[i]

            combined = f"{c}{vL}{vR}"
            a = get_hash(combined, self.p)

            c = c * pow(int(vL), a * a % self.p, self.q) % self.q * pow(int(vR), pow(a * a, -1, self.p), self.q) % self.q

            mid_point = len(g) // 2
            gL = g[:mid_point]
            gR = g[mid_point:]

            modL = np.vectorize(lambda x: pow(int(x), pow(a, -1, self.p), self.q))
            modR = np.vectorize(lambda x: pow(int(x), a, self.q))
            g = np.mod(modL(gL) * modR(gR), self.q)

        return pow(int(g[0]), int(final_x), self.q) == c

In [5]:
from random import randint

# Create a vector of size n from the random exponents of the generator
def create_public_params(generator, p, q, n):
    g = []
    for i in range(1, n + 1):
        g.append(pow(generator, randint(1, p - 1), q))
    return g

# Create a random vector of size n from {1, ..., p - 1}
def create_random_vector(p, n):
    x = []
    for i in range(1, n + 1):
        x.append(randint(1, p - 1))
    return x

# Check if g is a generator of a group of size p in the larger group of size q
# p and q should be prime and q = 2p + 1
# if q != 2p + 1, the order of group of g may be smaller than p but the function
# returns true!
def is_generator(g, q, p):
    return pow(g, p, q) == 1

# Find a generator of a group of size p in the larger group of size q
# p and q should be prime
def find_generator(q, p):
    while True:
        g = randint(1, q - 1)
        if is_generator(g, q, p):
            return g

def small_test_correct_commitment():
    # Commit to a vector of size 2
    # 3 is considered as the generator of a subgroup of size p (in group q)
    q = 11
    p = 5
    prover = Prover(p, q)
    verifier = Verifier(p, q)

    g = np.array([3, 9])
    x = np.array([1, 1])

    commitment = prover.commit(g, x)
    return verifier.verify_commitment(g, commitment)

def small_test_corrupted_commitment():
    # The test should fail because 2 and 4 are not generated from a generator
    # of a subgroup of size p
    q = 11
    p = 5
    prover = Prover(p, q)
    verifier = Verifier(p, q)

    g = np.array([2, 5])
    x = np.array([1, 1])
    commitment = prover.commit(g, x)

    # Corrupting the initial commitment
    # Its correct value is 10, we are altering it to 5
    commitment['initial_c'] = 5
    return verifier.verify_commitment(g, commitment)

def large_test_correct_commitment():
    # Commit to a vector of size 1024
    k = 10
    n = 2**k

    q = 1756823
    p = 878411
    prover = Prover(p, q)
    verifier = Verifier(p, q)

    genertor = find_generator(q, p)
    g = np.array(create_public_params(genertor, p, q, n))
    x = np.array(create_random_vector(p, n))

    commitment = prover.commit(g, x)
    return verifier.verify_commitment(g, commitment)

# Test 3 Scenarios
print(small_test_correct_commitment())
#assert(small_test_corrupted_commitment() == False)
#assert(large_test_correct_commitment() == True)

True
